#Ejemplo GPU
En este documento se evidencia la forma en la que se replica el ejemplo presentado por Sotaquirá en su video introductorio a Google Colab. Este cuaderno corresponde al caso en el que se usa la GPU para programar el modelo.

La preparación de datos y especificación es esencialmente igual que en el caso para CPU. Sin embargo es necesario, primero, establecer el entorno de ejecución en GPU.

In [1]:
import numpy as np
import os
import gzip
import tensorflow as tf

nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('GPU encontrada: {}'.format(nombre_gpu))



GPU encontrada: /device:GPU:0


In [2]:
def load_mnist(ruta, tipo='train'):

    ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo) 
    ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)

    with gzip.open(ruta_categorias, 'rb') as rut_cat:
     etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

    with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
     imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas),784)

    return imagenes, etiquetas



#Accediendo a Google Drive

In [29]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

ruta='gdrive/My Drive/Colab Notebooks/Tarea 1/'

X_train, Y_train = load_mnist(ruta, tipo='train')
X_test, Y_test = load_mnist(ruta, tipo='test')

Mounted at /content/gdrive


#Ajustando los datos

In [4]:
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

#Implementando el modelo

In [5]:
%tensorflow_version 2.x
print('Versión de TensorFlow: ' +tf.__version__)

tf.random.set_seed(200)

Versión de TensorFlow: 2.8.0


# Creación del modelo

In [24]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_6 (Batc  (None, 28, 28, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_7 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                      

In [25]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#Generando el Checkpoint

In [30]:
model_checkpoint_callback_GPU = tf.keras.callbacks.ModelCheckpoint(
    filepath='gdrive/My Drive/Colab Notebooks/Tarea 1/Codigos y checkpoints/Checkpoint GPU')

In [27]:
import timeit

def entrenamiento_gpu():
  with tf.device('/device:GPU:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1,callbacks=[model_checkpoint_callback_GPU])
  
  return None

gpu_time = timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu')

Epoch 1/2
468/468 [==============================] - 22s 46ms/step - loss: 0.6772 - accuracy: 0.7809 - val_loss: 0.3700 - val_accuracy: 0.8678
Epoch 2/2
468/468 [==============================] - 21s 45ms/step - loss: 0.3915 - accuracy: 0.8613 - val_loss: 0.3311 - val_accuracy: 0.8802
